In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

file_path = '../dataset_raw.csv'

try:
    df = pd.read_csv(file_path, encoding='ISO-8859-1')
    print(f"Berhasil memuat data dari: {file_path}")
except FileNotFoundError:
    print("File tidak ditemukan, mencoba folder saat ini...")
    df = pd.read_csv('dataset_raw.csv', encoding='ISO-8859-1')

df.head()

File tidak ditemukan, mencoba folder saat ini...


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [2]:
df_clean = df[(df['Quantity'] > 0) & (df['UnitPrice'] > 0)].copy()
df_clean = df_clean[df_clean['Quantity'] < 10000]
df_clean.dropna(subset=['CustomerID'], inplace=True)
df_clean = df_clean.drop_duplicates()
df_clean['InvoiceDate'] = pd.to_datetime(df_clean['InvoiceDate'])

print(f"Ukuran data setelah cleaning: {df_clean.shape}")

Ukuran data setelah cleaning: (392690, 8)


In [3]:
df_clean['Month_Year'] = df_clean['InvoiceDate'].dt.to_period('M')

monthly_sales = df_clean.pivot_table(
    index='StockCode',
    columns='Month_Year',
    values='Quantity',
    aggfunc='sum',
    fill_value=0
)

product_features = pd.DataFrame()
product_features['Avg_Sales'] = monthly_sales.mean(axis=1)
product_features['Std_Dev'] = monthly_sales.std(axis=1)
product_features['Max_Sales'] = monthly_sales.max(axis=1)

product_features['CV'] = product_features['Std_Dev'] / product_features['Avg_Sales']
product_features['CV'] = product_features['CV'].fillna(0)

product_info = df_clean.groupby('StockCode').agg({'UnitPrice': 'mean', 'Description': 'first'})
final_data = product_features.join(product_info)
final_data['Avg_Revenue'] = final_data['Avg_Sales'] * final_data['UnitPrice']

In [4]:
REV_Q3 = final_data['Avg_Revenue'].quantile(0.75)
MAX_Q3 = final_data['Max_Sales'].quantile(0.75)
CV_MEDIAN = final_data['CV'].median()

print(f"Threshold Omzet Tinggi (Revenue) > {REV_Q3:.2f}")
print(f"Threshold Stabil (CV) < {CV_MEDIAN:.2f}")

def quadrant_labeling(row):
    revenue = row['Avg_Revenue']
    max_sales = row['Max_Sales']
    volatility = row['CV']

    is_high_revenue = (revenue > REV_Q3) or (max_sales > MAX_Q3)
    is_stable = (volatility <= CV_MEDIAN)

    if is_high_revenue and is_stable:
        return 3

    elif is_high_revenue:
        return 2

    elif is_stable:
        return 1

    else:
        return 0

final_data['Label'] = final_data.apply(quadrant_labeling, axis=1)

print("\nSebaran 4 Label Baru:")
print(final_data['Label'].value_counts().sort_index())

Threshold Omzet Tinggi (Revenue) > 172.19
Threshold Stabil (CV) < 1.39

Sebaran 4 Label Baru:
Label
0    1436
1     982
2     396
3     850
Name: count, dtype: int64


In [5]:
final_data.reset_index().to_csv('data_training_final.csv', index=False)